In [1]:
import os

import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from utils import tabularModel, titanicDataset

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.suggest.bayesopt import BayesOptSearch

import time
%matplotlib inline

In [2]:
train_data = pd.read_csv(os.path.join('data', 'titanic', 'test.csv'))

In [3]:
train_data.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [4]:
def train(config):
    batch_size = int(config['batch_size'])
    num_embedding = int(config['num_embedding'])
    learning_rate = config['learning_rate']
    dataset = titanicDataset(os.path.join('/users', 'chang.liao', 'Projects', '4483A3', 'data', 'titanic', 'train.csv'))
    train_dataset, valid_dataset = torch.utils.data.random_split(dataset,
                                                                 [int(0.8 * len(dataset)),
                                                                  len(dataset) - int(0.8 * len(dataset))])
    train_dataloader = DataLoader(train_dataset,
                                  batch_size=batch_size, shuffle=True, num_workers=4)
    valid_dataloader = DataLoader(valid_dataset,
                                 batch_size=len(valid_dataset), shuffle=False, num_workers=4)
    categorical_columns = ['Pclass', 'Sex', 'Embarked', 'binned_family']
    categories = [3, 2, 3, 4]
    continuous_columns = ['Fare', 'Age']
    embedding_size = list(zip(categories, [num_embedding] * 4))
    model = tabularModel(embedding_size, categorical_columns, continuous_columns).double()
    optim = torch.optim.Adagrad(model.parameters(), lr=learning_rate, weight_decay=0.95)
    lossfunc = nn.CrossEntropyLoss()
    num_epoch = 300
    best_accu = 0
    cnt = 0
    model.train()
    for epoch in range(num_epoch):
        for i, (x, y) in enumerate(train_dataloader):
            optim.zero_grad()
            pred = model(x)
            loss = lossfunc(pred, y.squeeze().long())
            loss.backward()
            optim.step()

            if i % 10 == 0 or i == (len(train_dataset) / 16) - 1 :
                with torch.no_grad():
                    model.eval()
                    x, y = next(iter(valid_dataloader))
                    pred = model(x)
                    valid_loss = lossfunc(pred, y.squeeze().long())
                    accu = (pred.argmax(-1) == y.squeeze()).sum().item() / len(valid_dataset)
                # print("Epoch: {} | Train loss: {:.4f} | Valid loss: {:.4f} | Valid Accu: {:.4f}".format(
                #     epoch, loss.item(), valid_loss.item(), accu
                # ))
                model.train()
        # print("\n {:.4f} {:.4f} {} \n".format(1.02 * best_accu, accu, cnt))
        if accu >= best_accu:
            cnt = 0
            best_accu = accu
            # torch.save(model.state_dict(), os.path.join('model', 'torch_nn',
            #                                             'model-{}.pt'.format(epoch)))
        else:
            cnt += 1

        if cnt == 50:
            # print("Early stopped!")
            break
    return best_accu

In [5]:
def raytune_trainable(config, checkpoint_dir=None):
    tune.report(accu=train(config))

In [6]:
search_space = {
    'batch_size': (2, 64),
    'num_embedding': (4, 256),
    'learning_rate': (1e-4, 0.1)
}

In [29]:
algo = BayesOptSearch(space=search_space, metric='accu', mode='max', random_search_steps=10)
start = time.time()
analysis = tune.run(raytune_trainable, search_alg=algo, num_samples=300, 
                    # checkpoint_freq=4,
                    # checkpoint_at_end=True,
                    local_dir=os.curdir,
                    # resources_per_trial={'cpu': 9},
                    scheduler = ASHAScheduler(
                        metric='accu',
                        mode='max')
                   )
end = time.time()
print(end-start)

Trial name,status,loc,batch_size,learning_rate,num_embedding
raytune_trainable_cc14850a,RUNNING,,25.2215,0.0950764,188.462


2020-11-10 17:47:51,289	ERROR worker.py:1037 -- Possible unhandled error from worker: ray::ImplicitFunc.train() (pid=82963, ip=10.22.169.142)
  File "python/ray/_raylet.pyx", line 436, in ray._raylet.execute_task.function_executor
  File "/opt/anaconda3/lib/python3.8/site-packages/ray/tune/trainable.py", line 336, in train
    result = self.step()
  File "/opt/anaconda3/lib/python3.8/site-packages/ray/tune/function_runner.py", line 348, in step
    result = self._results_queue.get(
  File "/opt/anaconda3/lib/python3.8/queue.py", line 179, in get
    self.not_empty.wait(remaining)
  File "/opt/anaconda3/lib/python3.8/threading.py", line 306, in wait
    gotit = waiter.acquire(True, timeout)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

ray::ImplicitFunc.train() (pid=82963, ip=10.22.169.142)
  File "python/ray/_raylet.pyx", line 477, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 485, in ray._raylet.execute_task
ray.exceptions.

In [17]:
os.listdir(os.path.join('/users', 'chang.liao', 'Projects', '4483A3', 'data', 'titanic', 'train.csv'))

NotADirectoryError: [Errno 20] Not a directory: '/users/chang.liao/Projects/4483A3/data/titanic/train.csv'